In [53]:
import os
from glob import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from torch.utils.data import Dataset, DataLoader

import numpy as np
from PIL import Image

from tqdm import tqdm_notebook

In [54]:
seed = 1

lr = 0.001
momentum = 0.5

batch_size = 64
test_batch_size = 64

epochs = 5

no_cuda = False
log_interval = 100

### Model

In [55]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(5*5*50, 500)
        self.fc2 = nn.Linear(500, 10)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 5*5*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

### Preprocess

In [56]:
train_path = glob('../dataset/cifar/train/*.png')
test_path = glob('../dataset/cifar/test/*.png')
len(train_path), len(test_path)

(50000, 10000)

In [66]:
def get_label(path):
    lbl_name = os.path.basename(path).split('_')[-1].replace('.png','')
    label = np.argmax(classes == lbl_name)
    return label

class Dataset(Dataset):
    def __init__(self, data_paths, transform=None):
        self.data_paths = data_paths
        self.transform = transform
    
    def __len__(self):
        return len(self.data_paths)
    
    def __getitem__(self, idx):
        path = self.data_paths[idx]
        
        # Read Image
        image = Image.open(path)
        
         # Get Label
        label = get_label(path)        
        
        if self.transform:
            image = self.transform(image)
            
        return image, label

In [67]:
torch.manual_seed(seed)

use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')

kwargs = {'num_workers' : 1, 'pin_memory' : True} if use_cuda else {}

In [68]:
train_loader = torch.utils.data.DataLoader(
    Dataset(train_path,
            transforms.Compose([
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean = [0.406],
                    std = [0.225]
                )
            ])),
    batch_size=batch_size,
    shuffle=True,
    **kwargs
)

test_loader = torch.utils.data.DataLoader(
    Dataset(test_path,
            transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(
                    mean = [0.406],
                    std = [0.225]
                )
            ])),
    batch_size=batch_size,
    shuffle=False,
    **kwargs
)

### Optimizer

In [69]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(),
                      lr=lr,
                      momentum=momentum)

### Training

In [72]:
for epoch in range(1, 2):
    # Train mode
    model.train()
    
    for batch_idx, (data, target) in tqdm_notebook(enumerate(train_loader)):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad() # backpropagation 계산 전 0으로 기울기 초기화
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print('Train Epoch : {} [{} / {} ({:.0f}%)]\tLoss : {:.6f}'.format(
                epoch, batch_idx*len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()
            ))
            
    # Test mode
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(test_loader.dataset)
    
    print('\nTest set : Average loss : {:.4f}, Accuracy : {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)
    ))

<ipython-input-72-1750be2cb7e3>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_idx, (data, target) in tqdm_notebook(enumerate(train_loader)):


Train Epoch : 1 [0 / 50000 (0%)]	Loss : 2.303846
Train Epoch : 1 [6400 / 50000 (13%)]	Loss : 2.274155
Train Epoch : 1 [12800 / 50000 (26%)]	Loss : 2.254601
Train Epoch : 1 [19200 / 50000 (38%)]	Loss : 2.253160
Train Epoch : 1 [25600 / 50000 (51%)]	Loss : 2.196450
Train Epoch : 1 [32000 / 50000 (64%)]	Loss : 2.190099
Train Epoch : 1 [38400 / 50000 (77%)]	Loss : 2.086959
Train Epoch : 1 [44800 / 50000 (90%)]	Loss : 2.122360


Test set : Average loss : 2.0422, Accuracy : 2803/10000 (28%)



In [41]:
def foo(a, /, b, *, c):
    print("positional ", a)
    print("positional or keyword", b)
    print("keyword", c)
    
foo(40, 3.14, c = "hello")
print()
foo(40, b = 3.14, c = "hello")

positional  40
positional or keyword 3.14
keyword hello

positional  40
positional or keyword 3.14
keyword hello
